# Padoscope Footprint → Pressure Heatmap

padoscope로 획득한 RGB footprint와 기존 U-Net segmentation 결과(이진 마스크)를 입력으로 받아 정확한 족저압 히트맵을 회귀하는 PyTorch 파이프라인 예시입니다.

In [ ]:
import os
from dataclasses import dataclass
from pathlib import Path
from typing import Dict, List, Optional, Tuple

import numpy as np
from PIL import Image
import matplotlib.pyplot as plt

import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F
import torchvision.transforms as T
import torchvision.transforms.functional as TF

try:
    from torchmetrics.functional import structural_similarity_index_measure as ssim_fn
except Exception:
    ssim_fn = None
    print("torchmetrics가 설치되지 않았습니다. SSIM 손실 항은 비활성화됩니다.")

In [ ]:
@dataclass
class Config:
    root: Path = Path('data/padoscope')
    train_split: str = 'train.txt'
    val_split: str = 'val.txt'
    img_size: Tuple[int, int] = (256, 256)
    batch_size: int = 6
    num_workers: int = 4
    num_epochs: int = 120
    lr: float = 1e-4
    weight_decay: float = 1e-5
    input_channels: int = 4  # RGB*mask + mask
    output_channels: int = 1
    device: str = 'cuda' if torch.cuda.is_available() else 'cpu'
    ckpt_dir: Path = Path('output/padoscope')
    mean: Tuple[float, ...] = (0.485, 0.456, 0.406)
    std: Tuple[float, ...] = (0.229, 0.224, 0.225)
    ssim_weight: float = 0.1
    grad_weight: float = 0.25

cfg = Config()
cfg.ckpt_dir.mkdir(parents=True, exist_ok=True)
cfg